In [10]:
import pandas as pd
import json
import logging
import time
import backoff
import boto3
import pandas as pd


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 1000)

lambda_client = boto3.client("lambda")

@backoff.on_exception(backoff.expo, RuntimeError, max_time=15)
def call_lambda(payload):
    print(payload)
    response = lambda_client.invoke(
        FunctionName="Eleicoes2022-DataProcessi-GeoVotesFunctionCA5674F9-5OLJWl1KhVGr",
        InvocationType="Event",
        Payload=json.dumps(payload),
    )
    status_code = response['StatusCode']
    if status_code < 200 and status_code >= 300:
        raise RuntimeError

In [35]:
df = pd.read_csv(
    filepath_or_buffer="../data/enriched/votos_por_secao_com_cep.csv",
    encoding="utf-8",
    header=0,
    dtype={
        0: int,
        1: int,
        2: str,
        3: str,
        4: str,
        5: str,
        6: str,
        7: int,
        8: int,
        9: str,
        10: int,
        },
    )
df["CEP"] = df["CEP"].str[:-4]

df = df[["CEP", "TURNO", "NUMERO_CANDIDATO", "VOTOS"]]
df = df.groupby(["CEP", "TURNO", "NUMERO_CANDIDATO"]).sum()
df = df.reset_index()
total_rows = df.shape[0]

for row in df.itertuples():
    if row.CEP <= 2400:
        continue

    print(f"Processing row {row.Index} of {total_rows}")
    if row.VOTOS < 20000:
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": row.VOTOS}
        call_lambda(payload)
    
    elif row.VOTOS >= 20000 and row.VOTOS < 40000:
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": 19000}
        call_lambda(payload)
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": row.VOTOS - 19000}
        call_lambda(payload)
    
    elif row.VOTOS >= 40000 and row.VOTOS < 80000:
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": 20000}
        call_lambda(payload)
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": 19000}
        call_lambda(payload)
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": row.VOTOS - 39000}
        call_lambda(payload) 
    
    else:
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": 20000}
        call_lambda(payload)
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": 20000}
        call_lambda(payload)
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": 20000}
        call_lambda(payload)
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": 19000}
        call_lambda(payload)
        payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": row.VOTOS - 79000}
        call_lambda(payload) 
            
    time.sleep(row.VOTOS/10000)

    # Row 2294 possivelmente repetida

Processing row 2401 of 90479
{'cep': '0353', 'turno': 2, 'numero_candidato': 22, 'votos': 20000}
{'cep': '0353', 'turno': 2, 'numero_candidato': 22, 'votos': 19000}
{'cep': '0353', 'turno': 2, 'numero_candidato': 22, 'votos': 37535}
Processing row 2402 of 90479
{'cep': '0353', 'turno': 2, 'numero_candidato': 95, 'votos': 2954}
Processing row 2403 of 90479
{'cep': '0353', 'turno': 2, 'numero_candidato': 96, 'votos': 7201}
Processing row 2404 of 90479
{'cep': '0354', 'turno': 1, 'numero_candidato': 12, 'votos': 304}
Processing row 2405 of 90479
{'cep': '0354', 'turno': 1, 'numero_candidato': 13, 'votos': 2616}
Processing row 2406 of 90479
{'cep': '0354', 'turno': 1, 'numero_candidato': 15, 'votos': 562}
Processing row 2407 of 90479
{'cep': '0354', 'turno': 1, 'numero_candidato': 16, 'votos': 4}
Processing row 2408 of 90479
{'cep': '0354', 'turno': 1, 'numero_candidato': 21, 'votos': 2}
Processing row 2409 of 90479
{'cep': '0354', 'turno': 1, 'numero_candidato': 22, 'votos': 2480}
Process

KeyboardInterrupt: 

In [32]:
import boto3

sqs_client = boto3.client("sqs")

url='https://sqs.us-east-1.amazonaws.com/317456194246/Eleicoes2022-DataProcessing-GeoVotesFunctionDeadLetterQueueE7A009DE-9Ei3JDPcrg0o'
messages = sqs_client.receive_message(QueueUrl=url, MaxNumberOfMessages=10)["Messages"]

for message in messages:
    call_lambda(message["Body"])
    print(message)
    sqs_client.delete_message(
        QueueUrl=url,
        ReceiptHandle=message["ReceiptHandle"]
    )


{"cep": "0211", "turno": 1, "numero_candidato": 13, "votos": 290312}
{'MessageId': '68b93744-cc6a-4186-97c8-e89b4aa77309', 'ReceiptHandle': 'AQEBBrlNthCIjTgce32lWy5W2zySEdvnkKnINrdZU5tNTywe/gnAcpUvZ7tuTk7NqyL7MoHxiP0kORPaSfZg6v//Y9AIRSn5uXNWv73v8gwPHn01ECGQJNOItkDgfmoraRkMoMkvY4RjOB4vPdtJpcQNkqIUpDAoIyx9XBaZoWErjBDjQNpF5r3WrwSXtcTYfKOpEz15s6GL5CAsMMHa4EM+3vZncQn/vKbqdNh+Pz1f+p6CCy3tp0TtFpie2HJyfbuVwEG1LQVgjoAawfz2ct2pDr2vqlxTLTAkLHvfm1FPfnZR+AQgjwEtxgf28Lx2Hg0VdrVs8U4asnX0XNQkwMubB8RZpV47ETtQFoSV6WcI/PDBh8q9nE6wF/TjCKVcVOfzh/deusVrln8OgLt+Q/AL7yFhKzxYz3Krjjw2QVv9GSLSGaH/BAzlyfqOUa6oc3oqNs4v/yi39XL0FSXrQdsRLnqQmpNY+2bfwzLKYL1C9sI=', 'MD5OfBody': '00908e6693457e53b8e2d2f32eb620cc', 'Body': '{"cep": "0211", "turno": 1, "numero_candidato": 13, "votos": 290312}'}
{"cep": "0208", "turno": 1, "numero_candidato": 22, "votos": 27547}
{'MessageId': '3eefb2cd-ef73-4c77-9b18-1605b4c87885', 'ReceiptHandle': 'AQEB7XnlaJLH2ECaqLeo9ZNYcreKj67AGgxttMLSwJw82niW5vq366uLqosWC22YDf9XohMU/h5SjYoYfGgJ00HZyiV